In [17]:
import os 
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from time import sleep
import undetected_chromedriver as uc
import certifi
import os
import requests
import base64
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pytesseract
from PIL import Image, ImageFilter, ImageChops
from pathlib import Path
from pathlib import Path
import re
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from spacy.lang.ja.syntax_iterators import labels
from collections import Counter

def preprocess_image(image, img_width, img_height):
    image_resized = cv2.resize(image, (img_width, img_height))
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh_otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((2, 2), np.uint8)
    morph_img = cv2.morphologyEx(thresh_otsu, cv2.MORPH_CLOSE, kernel, iterations=2)
    dilated_img = cv2.dilate(morph_img, kernel, iterations=1)
    edges = cv2.Canny(dilated_img, 50, 150)

    return edges / 255.0 

def decode_prediction(pred, num_classes):
    pred_decoded = np.argmax(pred, axis=-1)    
    if pred_decoded.ndim > 1:
        pred_decoded = pred_decoded.flatten()
    
    return ''.join([str(p) for p in pred_decoded if p < num_classes])

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def tflite_predict(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data

def test_single_image(interpreter, img_path, img_width, img_height, num_classes):
    original_image = cv2.imread(img_path)    
    preprocessed_image = preprocess_image(original_image, img_width, img_height)

    X_test = np.array(preprocessed_image).reshape(img_width, img_height, 1)
    X_test = np.expand_dims(X_test, axis=0)
    
    prediction = tflite_predict(interpreter, X_test.astype(np.float32)) 
    predicted_label = decode_prediction(prediction, num_classes)
    return predicted_label

interpreter = tf.lite.Interpreter(model_path="Capmodel_098613.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [18]:
def captcha_solver(img):
    
    img_width, img_height = 150, 80
    num_classes = 10
    pred_decoded = test_single_image(interpreter, img, img_width, img_height, num_classes)    
    
    # Print prediction result
    print(f"Image: {os.path.basename(img)}, Predicted: {pred_decoded}")
    return pred_decoded
    


In [19]:
def captcha_handler(driver):
    try:
        driver.find_element(By.XPATH, '//h1[contains(text(), "403 Forbidden")]')
        print('Site access denied')
        driver.refresh()
        sleep(2)
    except:
        print('Site accessed')

    try:
        driver.find_element(By.XPATH, '//h1[contains(text(), "504 Gateway Time-out")]')
        print('Site access denied')
        driver.refresh()
        sleep(2)
    except:
        print('Site accessed')

    try:
        session_ex = driver.find_element(By.XPATH,
                                         '//div[contains(text(), "Your session has expired, please login again.")]')
        if session_ex.is_displayed():
            # login(driver)
            sleep(2)
            print('login successfully.....')
            driver.get('https://ita-pak.blsinternational.com/Global/bls/VisaTypeVerification')
    except:
        pass
        
    selected_img = 0
    
    # Find all div elements with the class "col-12 box-label"
    elements = driver.find_elements(By.CSS_SELECTOR,".col-12.box-label")
    
    # Iterate over each element and check if it is actually visible in the viewport
    for element in elements:
        # Use JavaScript to check if the element is in the viewport
        is_visible = driver.execute_script(
            "var elem = arguments[0],                 " +
            "  box = elem.getBoundingClientRect(),    " +
            "  cx = box.left + box.width / 2,         " +
            "  cy = box.top + box.height / 2,         " +
            "  e = document.elementFromPoint(cx, cy); " +
            "for (; e; e = e.parentElement) {         " +
            "  if (e === elem)                        " +
            "    return true;                         " +
            "}                                        " +
            "return false;", element)
        
        if is_visible:
            # Extract and print the visible text
            label_string = element.text
    
            # Regular expression to find the number
            number = re.search(r'\d+', label_string)
            
            if number:
                req_label = number.group()
                print('Target:', req_label)  # Output the extracted number
             
    # Find all img elements with class "captcha-img"
    images = driver.find_elements(By.CSS_SELECTOR, "img.captcha-img")
    
    # Iterate over each image and check if it is visible
    for index, img in enumerate(images):
        is_visible = driver.execute_script(
            "var elem = arguments[0], box = elem.getBoundingClientRect(), "
            "cx = box.left + box.width / 2, cy = box.top + box.height / 2, "
            "e = document.elementFromPoint(cx, cy); for (; e; e = e.parentElement) "
            "{ if (e === elem) return true; } return false;", img)
    
        if is_visible:
            # Get the image src (link)
            image_src = img.get_attribute("src")
    
            # If the image is in base64 format
            if image_src.startswith('data:image'):
                # Split the base64 header and data
                base64_data = image_src.split(',')[1]
    
                # Decode the base64 string
                image_data = base64.b64decode(base64_data)
    
                # Determine the file extension (set default as 'png')
                file_ext = 'png'
                file_path = f'captcha_image.{file_ext}'
                
                # Save the image in the captcha directory
                with open(file_path, 'wb') as handler:
                    handler.write(image_data)
    
                # Process the captcha and compare the result
                
                text = captcha_solver(str(file_path))
#                req_label_counter = Counter(req_label)
#                text_counter = Counter(text)

#                matches = sum(min(req_label_counter[char], text_counter[char]) for char in req_label_counter)
#                position_matches = sum(1 for i in range(min(len(req_label), len(text))) if req_label[i] == text[i]) or ((abs(int(req_label) - int(text))) <= 10 and (matches >= 2 or position_matches >= 2))

                if req_label == text :
                    img.click()
                    selected_img += 1

                    
                print(str(file_path),f" contains: {text}")
    if selected_img > 0:
        driver.find_element(By.CSS_SELECTOR, '[onclick="onSubmit();"]').click()
    else:
        driver.find_element(By.CSS_SELECTOR, '[onclick="onReload();"]').click()
        captcha_handler(driver)
        
    try:
        from selenium.webdriver.common.alert import Alert
        var = driver.switch_to.alert
        Alert(driver).accept()
    except:
        print('No Alert...')

In [20]:
import shutil
import tempfile

def get_driver():
    # Set the SSL_CERT_FILE environment variable to the path of the certifi bundle
    os.environ['SSL_CERT_FILE'] = certifi.where()

    options = uc.ChromeOptions()
    
    driver = uc.Chrome(options=options)
    driver.delete_all_cookies()


    # Return both driver and temp_dir so it can be deleted later
    return driver



In [21]:
driver = get_driver()
book_appointment_link = 'https://pakistan.blsspainglobal.com/Global/account/login'
driver.get(book_appointment_link)

In [22]:
while True:
    try:
        try:
            captcha_frame = driver.find_element(By.CSS_SELECTOR, '[title="Verify Registration"]')
        except:
            try:
                captcha_frame = driver.find_element(By.CSS_SELECTOR, '[title="Verify Selection"]')
            except:
                captcha_frame = driver.find_element(By.CSS_SELECTOR, '[title="Verify Appointment"]')
        
        driver.switch_to.frame(captcha_frame)
        captcha_handler(driver)
        driver.switch_to.default_content()
    
    except Exception as e:
        # You can uncomment the line below if you want to print the specific exception
        # print(e)
        driver.switch_to.default_content()
        print('No captcha found......')
        sleep(1)
        continue

    
    try:
        from selenium.webdriver.common.alert import Alert
        var = driver.switch_to.alert
        Alert(driver).accept()
    except:
        print('No Alert...')
    
    try:
        if driver.find_element(By.CSS_SELECTOR, '#btnVerified').is_displayed():
            print('User verified successfully...')
            break
    except:
        pass
    
    sleep(2)


No captcha found......
No captcha found......
No captcha found......
No captcha found......
Site accessed
Site accessed
Target: 632
Image: captcha_image.png, Predicted: 872
captcha_image.png  contains: 872
Image: captcha_image.png, Predicted: 334
captcha_image.png  contains: 334
Image: captcha_image.png, Predicted: 994
captcha_image.png  contains: 994
Image: captcha_image.png, Predicted: 751
captcha_image.png  contains: 751
Image: captcha_image.png, Predicted: 632
captcha_image.png  contains: 632
Image: captcha_image.png, Predicted: 599
captcha_image.png  contains: 599
Image: captcha_image.png, Predicted: 632
captcha_image.png  contains: 632
Image: captcha_image.png, Predicted: 632
captcha_image.png  contains: 632
Image: captcha_image.png, Predicted: 306
captcha_image.png  contains: 306
No Alert...
No Alert...
Site accessed
Site accessed
No captcha found......
No captcha found......
No captcha found......
No captcha found......
Site accessed
Site accessed
Target: 444
Image: captcha_ima

KeyboardInterrupt: 